# Accurate local problem

**Local goal:** create some procedure to determine learning rate scheduler for SGD.

$$
\tag{Loss}
f(\theta) \to \min_{\theta \in \mathbb{R}^p}
$$

$$
\tag{GF}
\dfrac{d \theta}{d t} = - \dfrac{1}{n} \sum_{i = 1}^s \nabla f_i (\theta)
$$
$n$ - the number of points, $s$ - the number of batches, $b*s = n, b$ - the batch size.


## Linear least squares (LLS)

$$
\tag{LLS.Loss}
f(\theta) = \dfrac{1}{n}  \|X\theta - y\|^2_2
$$

$$
\tag{LLS.GF}
\dfrac{d \theta}{d t} = - \dfrac{1}{n} \sum_{i = 1}^n x_i^*(x_i^*\theta - y_i)
$$

$\theta \in \mathbb{R}^p, X \in \mathbb{R}^{n \times p}, X_i \in \mathbb{R}^{b \times p}, y_i \in \mathbb{R}^b$

## Binary logistic regression (LogReg)

$$
\tag{LogReg.Loss}
f(\theta) = -\dfrac{1}{n} \sum\limits_{i=1}^n \left( y_i \ln h_\theta(x_i) + (1-y_i) \ln (1- h_\theta(x_i))\right)
$$

$$
\tag{LogReg.GF}
\dfrac{d \theta}{d t} = - \dfrac{1}{n} \sum\limits_{i=1}^n x_i^* (h_\theta(x_i^*) - y_i)
$$

$h_\theta(x_i) = \dfrac{1}{1 + e^{-\theta^*x_i}}$ - the sigmoid function, $\theta \in \mathbb{R}^p, X \in \mathbb{R}^{n \times p}, X_i \in \mathbb{R}^{b \times p}, y_i \in \mathbb{R}^b$

# Approach

Each batch gradient (or, stochastic gradient) has rank $b$. In case of unit batchsize, we have rank one stochastic gradient.

We will search for some scalar function $g (\eta)$, s.t.
$$
\tag{Approach}
\dfrac{d \theta}{d t} \sim q_i \cdot g(\eta_i), \quad
$$

where $q_i$ is the basis vector of some low rank subspace of $i$-th batch gradient of batch of the number $i$.

# Code

## Softmax on MNIST

In [0]:
import torchvision.datasets as datasets
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
import copy
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class LogisticRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(int(input_dim), int(output_dim))
    def forward(self, x):
    	x = x.contiguous().view(x.size(0), -1)
    	out = F.softmax(self.linear(x), dim=1)
    	return out

def load_model(X_test, y_test):
	'''
	Returns logistic regression model
	Which is just single linear layer with flattening at the beginning and softmax at the end
	'''
	input_dim = X_test[0].numel()

	# Handling with usual and one-hot format
	if any(y_test > 1):
		output_dim = max(y_test) + 1
	else:
		output_dim = 2

	model = LogisticRegression(input_dim, output_dim)
	return model

def load_batched_data(batch_size=50, shuffle = True):
	'''
	Load batches of MNIST data.

	Output: X_trains - N_train batches of training data, 
			y_trains - N_train batches of labels,
			X_test - test points
			y_test - test labels
	X_trains: torch.array of shape (N_train,batch_size,*X_train[0].shape),
		where 
		N_train - the number of batches, 
		batch_size - batch size
		*X_train[0].shape - shape of the dataset point;

	y_trains: torch.array of shape (N_train,batch_size);

	X_test: torch.array of shape (s_test,*X_train[0].shape),
		where
		s_test - the number of test points;

	y_test: torch.array of shape (s_test);
	'''

	trainset = datasets.MNIST('./mnist_data/', download=True, train=True)
	X_train = trainset.data.to(dtype=torch.float)/255
	y_train = trainset.targets
	mask    = y_train <=1
	X_train = X_train[mask]
	y_train = y_train[mask]
	X_train.resize_(len(X_train), 1, *X_train[0].shape)
	y_train.view(-1).long()

	if shuffle == True:
		shuffling = torch.randperm(len(y_train))
		X_train = X_train[shuffling]
		y_train = y_train[shuffling]

	# Download and load the test data
	testset = datasets.MNIST('./mnist_data/', download=True, train=False)
	X_test = testset.data.to(dtype=torch.float)/255
	y_test = testset.targets
	mask   = y_test <=1
	X_test = X_test[mask]
	y_test = y_test[mask]
	X_test.resize_(len(X_test), 1, *X_test[0].shape)
	y_test.view(-1).long()

	if shuffle == True:
		shuffling = torch.randperm(len(y_test))
		X_test = X_test[shuffling].to(device)
		y_test = y_test[shuffling]

	s_train = len(y_train)
	s_test  = len(y_test)

	N_train = int(s_train/batch_size)   # Number of training batches

	X_trains = torch.zeros((N_train,batch_size,*X_train[0].shape),requires_grad=False).to(device)
	y_trains = torch.zeros((N_train,batch_size),requires_grad=False, dtype=torch.int64).to(device)

	for i in range(N_train):
	    X_trains[i] = X_train[batch_size*i:batch_size*(i+1), :]
	    y_trains[i] = y_train[batch_size*i:batch_size*(i+1)]

	return X_trains, y_trains, X_test, y_test

def train(model, X_trains, y_trains, X_test, y_test, n_epochs, hs, weights_name = './'):
	model     = model.to(device)
	criterion = nn.CrossEntropyLoss()
	N_batches  = int(X_trains.size(0))
	batch_size = int(X_trains.size(1)) 

	history = {}  
	history['train_loss'] = np.zeros(n_epochs)
	history['test_loss']  = np.zeros(n_epochs)
	history['train_acc']  = np.zeros(n_epochs)
	history['test_acc']   = np.zeros(n_epochs)

	for epoch in tqdm(range(n_epochs)):
		#==== Forward steps  ====
		model.train()
		h = hs[epoch]
		for i_batch in range(N_batches):
			# Forward pass
			images, labels = X_trains[i_batch], y_trains[i_batch]
			images, labels = images.to(device), labels.to(device)
			predictions = model(images)
			loss = criterion(predictions, labels)

			# Gradient step
			model.zero_grad()
			loss.backward()
			# torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
			for idp, parameter in enumerate(model.parameters()):
				grad           = parameter.grad.data
				grad           = grad.to(device)
				parameter.data = parameter.data - h*grad

		# Metrics calculation
		history['train_loss'][epoch] = loss.data
		pred_label = torch.max(predictions, 1, keepdim=True)[1]
		history['train_acc'][epoch] = pred_label.eq(labels.data.view_as(pred_label)).sum().to(dtype=torch.float)/len(labels)
		with torch.no_grad():
			model.eval()
			Xst = X_test.to(device)
			yst = y_test.to(device)
			predictions = model(Xst)
			loss = criterion(predictions, yst)
			history['test_loss'][epoch] = loss.data
			pred_label = torch.max(predictions, 1, keepdim=True)[1]
			history['test_acc'][epoch] = pred_label.eq(yst.data.view_as(pred_label)).sum().to(dtype=torch.float)/len(yst)
	return history

def weight_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight.data)

def model_init(model, parameter_list):
	new_model = copy.deepcopy(model)
	for idp, parameter in enumerate(new_model.parameters()):
		parameter.data = parameter_list[idp].to(device)
	return new_model

def vector_of_parameters(model, parameter_column):
	params = list(model.parameters())
	W, b = params
	return W[parameter_column, :]

def calculate_batch_gradient(model, X_train, y_train, n, parameter_column):
	model     = model.to(device)
	criterion = nn.CrossEntropyLoss()
	batch_size  = int(X_train.size(0))
	model.train()
	images, labels = X_train.to(device), y_train.to(device)
	predictions = model(images)
	loss = criterion(predictions, labels)

	# Gradient step
	model.zero_grad()
	loss.backward()

	grads = []
	normi = -batch_size/n
	for parameter in model.parameters():
		grads.append(normi*parameter.grad.data)

	vec_grad = torch.Tensor().to(device)
	W_grad, b_grad = grads
	vec_grad = W_grad[parameter_column, :]
	return vec_grad

def calculate_main_singular_vector(model, X_train, y_train, n, parameter_column, N_points = 100):
	batch_gradient = []
	parameter_list = list(model.parameters())
	parameter_list_corrupted = [0 for i in range(len(parameter_list))] # Duplication of length
	for i_gradient in range(N_points):
		for idp, parameter in enumerate(parameter_list):
			parameter_list_corrupted[idp] = parameter_list[idp] + 1e-2*torch.rand(*parameter_list[idp].shape).to(device)
		model_corr = model_init(model, parameter_list_corrupted)
		batch_gradient.append(calculate_batch_gradient(model_corr, X_train, y_train, n, parameter_column))
	batch_gradient_matrix = torch.stack(batch_gradient).transpose(0,1)
	u, s, v = torch.svd(batch_gradient_matrix)
	return u[:, 0]

def plot_projection_picture(model, X_train, y_train, n, parameter_column, N_points = 100, plot = True):
	q = calculate_main_singular_vector(model, X_train, y_train, n, parameter_column)
	# q = X_train.view(-1)
	parameter_list = list(model.parameters())
	parameter_list_corrupted = [0 for i in range(len(parameter_list))] # Duplication of length
	Pi_q_theta = []
	Pi_q_grads = []

	for i in range(N_points):
		for idp, parameter in enumerate(parameter_list):
			parameter_list_corrupted[idp] = parameter_list[idp] + 1e-2*torch.rand(*parameter_list[idp].shape).to(device)
		model_corr = model_init(model, parameter_list_corrupted)
		theta = vector_of_parameters(model_corr, parameter_column)
		grad = calculate_batch_gradient(model_corr, X_train, y_train, n, parameter_column)

		Pi_q_theta.append(torch.dot(q, theta))
		Pi_q_grads.append(torch.dot(q, grad))
  
	if plot == False:
		return np.array(Pi_q_theta), np.array(Pi_q_grads)

	plt.title('Softmax regression on MNIST') 
	plt.xlabel(r'$\Pi_q(\theta)$')
	plt.ylabel(r'$\Pi_q(\nabla f_i)$')
	plt.plot(Pi_q_theta, Pi_q_grads, 'bx')
	plt.show()
	return None

In [0]:
batch_size = 64
X_trains, y_trains, X_test, y_test = load_batched_data(batch_size=batch_size)
n = y_trains.shape[0]*y_trains.shape[1]
N_batches = n//batch_size
model = load_model(X_test, y_test)
model.apply(weight_init)

n_epochs = 6
h = 0.005
train(model, X_trains, y_trains, X_test, y_test, n_epochs, [h for i in range(n_epochs)])







  0%|          | 0/6 [00:00<?, ?it/s]





 17%|█▋        | 1/6 [00:00<00:00,  6.04it/s]





 33%|███▎      | 2/6 [00:00<00:00,  5.91it/s]





 50%|█████     | 3/6 [00:00<00:00,  5.75it/s]





 67%|██████▋   | 4/6 [00:00<00:00,  5.50it/s]





 83%|████████▎ | 5/6 [00:00<00:00,  5.44it/s]





100%|██████████| 6/6 [00:01<00:00,  5.35it/s]







{'test_acc': array([0.99810874, 0.99810874, 0.99810874, 0.99810874, 0.99810874,
        0.99810874]),
 'test_loss': array([0.36985454, 0.34492531, 0.33591232, 0.33119485, 0.32826746,
        0.3262617 ]),
 'train_acc': array([1., 1., 1., 1., 1., 1.]),
 'train_loss': array([0.37143812, 0.34732535, 0.33852497, 0.33385476, 0.33091587,
        0.32887518])}

In [0]:
parameter_column = 0
i_batch = 0
Pi_q_theta, Pi_q_grads = plot_projection_picture(model, X_trains[i_batch], y_trains[i_batch], n, parameter_column, plot=False)
for i_batch in tqdm(range(1, N_batches)):
    Pi_q_theta_new, Pi_q_grads_new = plot_projection_picture(model, X_trains[i_batch], y_trains[i_batch], n, parameter_column, plot=False)
    Pi_q_theta = (i_batch-1)/i_batch* Pi_q_theta + 1/i_batch * Pi_q_theta_new
    Pi_q_grads = (i_batch-1)/i_batch* Pi_q_grads + 1/i_batch * Pi_q_grads_new

In [0]:
plt.title('Softmax regression on MNIST') 
plt.xlabel(r'$\Pi_q(\theta)$')
plt.ylabel(r'$\Pi_q(\nabla f_i)$')
plt.plot(Pi_q_theta, Pi_q_grads, 'bx')
plt.show()

NameError: ignored

In [0]:
y_trains.shape

torch.Size([395, 32])